In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Charger les données depuis le fichier CSV
data = pd.read_csv('C:/Users/hp/Desktop/prj_pfe_inwi/traffic_matrix.csv')
def diviser_table(data):
    n = len(data)
    quart = n // 2  # Nombre d'éléments dans chaque sous-table

    data1 = data[:quart]
    data2 = data[quart:2*quart]

    return data1, data2

# Exemple d'utilisation

data1, data2= diviser_table(data)

# Afficher les sous-tables
print("data1:", data1)
print("data2:", data2)



data1:                                 a_number  a_own  \
0       805249d5353b56beffe265acb21d2922   True   
1       805249d5353b56beffe265acb21d2922   True   
2       805249d5353b56beffe265acb21d2922   True   
3       805249d5353b56beffe265acb21d2922   True   
4       805249d5353b56beffe265acb21d2922   True   
...                                  ...    ...   
199995  69061142d86f77340206b6cd9c3778a3   True   
199996  62e48a886921108f43011b2ea85bcedb   True   
199997  62e48a886921108f43011b2ea85bcedb   True   
199998  62e48a886921108f43011b2ea85bcedb   True   
199999  62e48a886921108f43011b2ea85bcedb   True   

                                b_number  duration  duration_wh  \
0       c0cd71c4d34d274eca6aa7d3be7f66a4  9.223849     7.893572   
1       f849a35b994ebbde78d26aa7e507a589  6.948897     6.047372   
2       f7bbf080d6d62e98b98784957f31ee37  8.730852     7.926964   
3       ca215256553a2ba6e75728c224658482  4.418841     0.000000   
4       62a248714a24f84a59ab89a9e07866be  1.3

In [2]:
data = data1[data1['a_own'] == True]


In [3]:
columns=['duration', 'duration_wh',
       'num_interactions', 'num_interactions_wh', 'num_interactions_sms',
       'num_interactions_sms_wh']

In [4]:
# Filtrer les colonnes catégorielles de type 'object'
colonnes_categorielles = data.select_dtypes(include=['object'])

# Afficher les colonnes catégorielles
print(colonnes_categorielles.columns)

Index(['a_number', 'b_number'], dtype='object')


In [14]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for colonne in colonnes_categorielles.columns:
    # Encoder la variable catégorielle
    encoded_variable = label_encoder.fit_transform(data[colonne])
    # Assigner les valeurs encodées à la colonne correspondante dans la dataframe
    data[colonne] = encoded_variable
    

C:\Users\hp\AppData\Local\Temp\ipykernel_12532\1186893553.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[colonne] = encoded_variable
C:\Users\hp\AppData\Local\Temp\ipykernel_12532\1186893553.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[colonne] = encoded_variable


In [5]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[columns])

In [6]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from itertools import product

In [7]:
# Paramètres à tester pour DBSCAN
min_samples_values = [3, 5, 7]  # Vous pouvez ajuster ces valeurs en fonction de votre taille de données
eps_values = [0.5, 1.0, 1.5]    # Vous pouvez ajuster ces valeurs en fonction de votre échelle de données


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
# Variables pour stocker les meilleures valeurs
best_eps = None
best_min_samples = None
best_score = -1
best_num_clusters = 0

# Recherche des valeurs optimales
for eps, min_samples in product(eps_values, min_samples_values):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = dbscan.fit_predict(data_scaled)
    
    # Ignorer les résultats où tous les points sont considérés comme du bruit (-1)
    if len(np.unique(clusters)) > 1:
        num_clusters = len(np.unique(clusters)) - 1  # Soustraire 1 pour ne pas compter le bruit comme un cluster
        if 3 <= num_clusters <= 10:
            silhouette_avg = silhouette_score(data_scaled, clusters)
            if silhouette_avg > best_score:
                best_eps = eps
                best_min_samples = min_samples
                best_score = silhouette_avg
                best_num_clusters = num_clusters

# Affichage des valeurs optimales
print("Valeurs optimales :")
print("eps =", best_eps)
print("min_samples =", best_min_samples)
print("best_num_clusters =", best_num_clusters)

print(f"Silhouette moyenne: {silhouette_avg}")

In [ ]:

# Appliquer DBSCAN avec les meilleures valeurs d'epsilon et de min_samples
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
clusters = dbscan.fit_predict(data_scaled)

# Ajouter les informations de cluster au DataFrame original
data['cluster'] = clusters

# Afficher le nombre de clusters créés et les échantillons dans chaque cluster
num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
num_outliers = list(clusters).count(-1)
print("Nombre de clusters : ", num_clusters)
print("Nombre d'échantillons aberrants : ", num_outliers)

# Afficher les données avec les clusters
print(data.head())





In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Afficher le scatter plot 3D des résultats du clustering
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Données de clustering
# Assurez-vous d'avoir les données 'data', les étiquettes de cluster 'clusters', et les colonnes 'columns' définies

# Définir les couleurs pour chaque cluster
colors = ['blue', 'red', 'green', 'orange', 'purple', 'yellow', 'cyan', 'magenta', 'lime', 'pink', 'brown', 'teal', 'lavender', 'olive', 'gold', 'salmon', 'lightblue', 'darkgreen', 'violet', 'lightgreen', 'coral', 'skyblue']

# Obtenez la liste des étiquettes de cluster uniques
cluster_labels = set(clusters)

for cluster_label in cluster_labels:
    if cluster_label == -1:
        # Echantillons aberrants (outliers) sont affichés en noir
        cluster_data = data[data['cluster'] == cluster_label][columns]
        ax.scatter(cluster_data[columns[0]], cluster_data[columns[1]], cluster_data[columns[2]], color='black', alpha=0.5, label='Outliers')
    else:
        # Echantillons du cluster sont affichés dans une couleur spécifique
        cluster_data = data[data['cluster'] == cluster_label][columns]
        color_index = list(cluster_labels).index(cluster_label) % len(colors)
        cluster_color = colors[color_index]
        ax.scatter(cluster_data[columns[0]], cluster_data[columns[1]], cluster_data[columns[2]], color=cluster_color, alpha=0.5, label='Cluster {}'.format(cluster_label))

ax.set_xlabel(columns[0])
ax.set_ylabel(columns[1])
ax.set_zlabel(columns[2])
ax.set_title('DBSCAN Clustering')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Afficher le scatter plot 3D des résultats du clustering
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Données de clustering
# Assurez-vous d'avoir les données 'data', les étiquettes de cluster 'clusters', et les colonnes 'columns' définies

# Définir les couleurs pour chaque cluster
colors = ['blue', 'red', 'green', 'orange', 'purple', 'yellow', 'cyan', 'magenta', 'lime', 'pink', 'brown', 'teal', 'lavender', 'olive', 'gold', 'salmon', 'lightblue', 'darkgreen', 'violet', 'lightgreen', 'coral', 'skyblue']

# Obtenez la liste des étiquettes de cluster uniques
cluster_labels = set(clusters)

for cluster_label in cluster_labels:
    if cluster_label == -1:
        # Echantillons aberrants (outliers) sont affichés en noir
        cluster_data = data[data['cluster'] == cluster_label][columns]
        ax.scatter(cluster_data[columns[3]], cluster_data[columns[4]], cluster_data[columns[2]], color='black', alpha=0.5, label='Outliers')
    else:
        # Echantillons du cluster sont affichés dans une couleur spécifique
        cluster_data = data[data['cluster'] == cluster_label][columns]
        color_index = list(cluster_labels).index(cluster_label) % len(colors)
        cluster_color = colors[color_index]
        ax.scatter(cluster_data[columns[3]], cluster_data[columns[4]], cluster_data[columns[2]], color=cluster_color, alpha=0.5, label='Cluster {}'.format(cluster_label))

ax.set_xlabel(columns[3])
ax.set_ylabel(columns[4])
ax.set_zlabel(columns[2])
ax.set_title('DBSCAN Clustering')

plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()


In [ ]:
# Afficher le scatter plot 2D des résultats du clustering
plt.figure(figsize=(8, 6))

colors = ['blue', 'red', 'green', 'orange', 'purple', 'yellow', 'cyan', 'magenta', 'lime', 'pink', 'brown', 'teal', 'lavender', 'olive', 'gold', 'salmon', 'lightblue']
 # Couleurs pour chaque cluster

for cluster_label in set(clusters):
    if cluster_label == -1:
        # Echantillons aberrants (outliers) sont affichés en noir
        cluster_data = data[data['cluster'] == cluster_label][columns]
        plt.scatter(cluster_data[columns[0]], cluster_data[columns[1]], color='black', alpha=0.5, label='Outliers')
    else:
        # Echantillons du cluster sont affichés dans une couleur spécifique
        cluster_data = data[data['cluster'] == cluster_label][columns]
        plt.scatter(cluster_data[columns[0]], cluster_data[columns[1]], color=colors[cluster_label % len(colors)], alpha=0.5, label='Cluster {}'.format(cluster_label))

plt.xlabel(columns[0])
plt.ylabel(columns[1])
plt.title('DBSCAN Clustering')

# Afficher la légende
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()